In [15]:
import timm
import torch
from torch import nn
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [17]:
def duplicate_by_symmetry(x):
    result = []
    for row in x:
        num_of_features = int((len(row) - 1) / 2)
        left = row[:num_of_features]
        right = row[num_of_features:-1]
        label = row[-1]
        result.append(np.concatenate((left, right, [label])))
        result.append(np.concatenate((right, left, [label])))
    return np.array(result)

# Load from .npy
dev_train = duplicate_by_symmetry(np.load('dev-train.npy'))
dev_val = np.load('dev-val.npy')
dev_train.shape, dev_val.shape

((4400, 1025), (1000, 1025))

In [18]:
X_train = dev_train[:, :-1]
y_train = dev_train[:, -1]
X_val = dev_val[:, :-1]
y_val = dev_val[:, -1]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((4400, 1024), (4400,), (1000, 1024), (1000,))

In [19]:
### Regressão logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


c:\Python312\Lib\site-packages\sklearn\utils\optimize.py:99: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
c:\Python312\Lib\site-packages\sklearn\utils\optimize.py:311: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
c:\Python312\Lib\site-packages\sklearn\utils\optimize.py:99: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
c:\Python312\Lib\site-packages\sklearn\utils\optimize.py:311: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Best parameters:  {'C': 0.0031622776601683794, 'solver': 'lbfgs'}
Best cross-validation score:  0.5836363636363636


In [20]:

logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))


Train accuracy: 0.6827272727272727
Val accuracy: 0.6


In [21]:

### Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)


Best parameters:  {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': None}
Best cross-validation score:  0.8940909090909092


In [22]:

rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))


Train accuracy: 0.9943181818181818
Val accuracy: 0.863


In [23]:

### Rede neural
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs2',
    project_name=f'nn-timm-siamese-mediumd')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]


Trial 5 Complete [00h 00m 34s]
val_accuracy: 0.870800006389618

Best val_accuracy So Far: 0.8802000045776367
Total elapsed time: 00h 02m 42s
Results summary
Results in runs2\nn-timm-siamese-mediumd
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 16
learning_rate: 0.001
units_1: 4
Score: 0.8802000045776367

Trial 3 summary
Hyperparameters:
num_layers: 2
units_0: 20
learning_rate: 0.01
units_1: 4
Score: 0.8757999897003174

Trial 4 summary
Hyperparameters:
num_layers: 2
units_0: 20
learning_rate: 0.0001
units_1: 16
Score: 0.870800006389618

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 16
learning_rate: 0.0001
units_1: 28
Score: 0.8692000031471252

Trial 2 summary
Hyperparameters:
num_layers: 1
units_0: 4
learning_rate: 0.0001
units_1: 12
Score: 0.8085999965667725


In [24]:

best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()


Epoch 1/40


138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7135 - loss: 0.5398 - val_accuracy: 0.8200 - val_loss: 0.4138
Epoch 2/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.8758 - loss: 0.2903 - val_accuracy: 0.8470 - val_loss: 0.3746
Epoch 3/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.8984 - loss: 0.2507 - val_accuracy: 0.8490 - val_loss: 0.3669
Epoch 4/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.9157 - loss: 0.2177 - val_accuracy: 0.8390 - val_loss: 0.3790
Epoch 5/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.9209 - loss: 0.2130 - val_accuracy: 0.8430 - val_loss: 0.4090
Epoch 6/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.9372 - loss: 0.1857 - val_accuracy: 0.8590 - val_loss: 0.3636
Epoch 7/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.9411 - loss: 0.1777 - val_accuracy: 0.8500 - val_loss: 0.3698
Epoch 8/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.9393 - loss: 0.1572 - val_accuracy:

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │        16,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            68 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,421 (193.05 KB)

 Trainable params: 16,473 (64.35 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 32,948 (128.71 KB)

In [25]:

train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.9996 - loss: 0.0080
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.8816 - loss: 0.4925


(0.9986363649368286, 0.8700000047683716)

In [26]:


### SVM
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score:  0.9168181818181818


In [27]:

# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 0.9963636363636363
Val accuracy: 0.875
